In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import np_utils

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\jefferson\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
Tweets = pd.read_csv("./twitters/Tweets2.csv")
Tweets.shape

(74682, 4)

In [4]:
Tweets.head()

,id,local,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [5]:
Tweets.groupby(['sentiment']).size()

sentiment
Irrelevant    12990
Negative      22542
Neutral       18318
Positive      20832
dtype: int64

In [6]:
Tweets.loc[Tweets['sentiment']=='Irrelevant','sentiment'] = 'Neutral'

In [7]:
Tweets = Tweets.dropna(subset=['text'])
Tweets.reset_index(drop=True, inplace=True)

In [8]:
Tweets.shape

(73996, 4)

# **Supervisionado**

In [9]:
token = Tokenizer(num_words=100)
token.fit_on_texts(Tweets['text'].values)

In [10]:
X = token.texts_to_sequences(Tweets['text'].values)
X = pad_sequences(X, padding="post", maxlen=100)

In [11]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(Tweets['sentiment'])
print(y)

[2 2 2 ... 2 2 2]


In [12]:
y = np_utils.to_categorical(y)
print(y)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.4)
X_test

array([[ 9,  3, 50, ...,  0,  0,  0],
       [ 2, 23,  1, ...,  0,  0,  0],
       [39, 31,  0, ...,  0,  0,  0],
       ...,
       [ 2,  1,  0, ...,  0,  0,  0],
       [40,  5,  0, ...,  0,  0,  0],
       [14,  6,  0, ...,  0,  0,  0]])

In [14]:
modelo = Sequential()
modelo.add(Embedding(input_dim= len(token.word_index), output_dim=128, input_length=X.shape[1]))
modelo.add(SpatialDropout1D(0.2))
modelo.add(LSTM(units=196, dropout=0.2, recurrent_dropout=0, activation='tanh',
                recurrent_activation='sigmoid', unroll=False, use_bias=True))
modelo.add(Dense(units=3,activation="softmax"))

In [15]:
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(modelo.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          4324224   
                                                                 
 spatial_dropout1d (SpatialD  (None, 100, 128)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 3)                 591       
                                                                 
Total params: 4,579,615
Trainable params: 4,579,615
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
modelo.fit(X_train, y_train, epochs=100, batch_size=500,verbose=True)

Epoch 1/100
89/89 [==============================] - 14s 44ms/step - loss: 1.0820 - accuracy: 0.4204
Epoch 2/100
89/89 [==============================] - 4s 43ms/step - loss: 1.0814 - accuracy: 0.4224
Epoch 3/100
89/89 [==============================] - 4s 43ms/step - loss: 1.0813 - accuracy: 0.4224
Epoch 4/100
89/89 [==============================] - 4s 42ms/step - loss: 1.0815 - accuracy: 0.4224
Epoch 5/100
89/89 [==============================] - 4s 41ms/step - loss: 1.0812 - accuracy: 0.4224
Epoch 6/100
89/89 [==============================] - 4s 43ms/step - loss: 1.0813 - accuracy: 0.4224
Epoch 7/100
89/89 [==============================] - 4s 43ms/step - loss: 1.0813 - accuracy: 0.4224
Epoch 8/100
89/89 [==============================] - 4s 44ms/step - loss: 1.0812 - accuracy: 0.4224
Epoch 9/100
89/89 [==============================] - 4s 50ms/step - loss: 1.0812 - accuracy: 0.4224
Epoch 10/100
89/89 [==============================] - 4s 43ms/step - loss: 1.0814 - accuracy: 0.422

In [17]:
_, accuracy = modelo.evaluate(X_test,y_test)
print("Accuracy: ", accuracy)

925/925 [==============================] - 8s 7ms/step - loss: 1.0844 - accuracy: 0.4132
Accuracy:  0.41322341561317444


# **VADER**

In [18]:
#from pandas.core.internals.managers import T
mas = SentimentIntensityAnalyzer()
Tweets['vander_sentiment'] = ''

for y in range(len(Tweets.index)):
  x = mas.polarity_scores(Tweets['text'].iloc[y])
  del x['compound']
  maior = max(x,key=x.get) #neg pos neu
  Tweets.loc[y,'vander_sentiment'] = maior

In [19]:
Tweets.groupby(['vander_sentiment']).size()

vander_sentiment
neg     3660
neu    65581
pos     4755
dtype: int64

In [20]:
Tweets.groupby(['sentiment']).size()

sentiment
Negative    22358
Neutral     30983
Positive    20655
dtype: int64

In [21]:
Tweets.loc[Tweets['vander_sentiment']== 'neu' , 'vander_sentiment'] = 'Neutral'
Tweets.loc[Tweets['vander_sentiment']== 'neg' , 'vander_sentiment'] = 'Negative'
Tweets.loc[Tweets['vander_sentiment']== 'pos' , 'vander_sentiment'] = 'Positive'

In [22]:
Tweets.groupby(['vander_sentiment']).size()

vander_sentiment
Negative     3660
Neutral     65581
Positive     4755
dtype: int64

In [23]:
y_pred = Tweets['vander_sentiment']
y_test = Tweets['sentiment']
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 2004 19902   452]
 [ 1122 28384  1477]
 [  534 17295  2826]]


In [24]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.44886210065408944
